In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from pyproj import Transformer
import networkx as nx

from bokeh.io import output_file, show
from bokeh.models import Range1d, Slider, CustomJS, ColumnDataSource, BooleanFilter, CDSView, Legend, LegendItem, HoverTool, StaticLayoutProvider
from bokeh.plotting import figure, from_networkx
from bokeh.layouts import column, row
from bokeh.models import Circle, MultiLine, Div
from bokeh.transform import linear_cmap
from bokeh.models.glyphs import Rect
from bokeh.tile_providers import get_provider, Vendors

from matplotlib import cm
import matplotlib.colors as mcolors

def lonlat_to_mercator(lon, lat):
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    x, y = transformer.transform(lon, lat)
    return x, y

def get_lat_lon(city):
    try:
        location = geolocator.geocode(city)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None
    

geolocator = Nominatim(user_agent="artvis_project")

data = pd.read_csv(r'C:\TUW\courses\Vis\project\artvis\data\artvis_cleaned.csv')
output_path = r"C:\TUW\courses\Vis\project\artvis\pages\exhibition_network.html"

top_20_artist_ids = [401, 317, 225, 58, 517, 467, 65, 37, 276, 461, 268, 581, 5, 905, 1708, 164, 412, 477, 514, 413]
data = data[data['a.id'].isin(top_20_artist_ids)]

data['a.lat'], data['a.lon'] = zip(*data['a.birthplace'].apply(
    lambda city: get_lat_lon(city) if pd.notna(city) else (None, None)))
data['a.x'], data['a.y'] = zip(*data[['a.lon', 'a.lat']].apply(
    lambda row: lonlat_to_mercator(row['a.lon'], row['a.lat']) if pd.notna(row['a.lon']) and pd.notna(row['a.lat']) else (None, None), axis=1))

# data['e.lat'], data['e.lon'] = zip(*data['e.city'].apply(
#     lambda city: get_lat_lon(city) if pd.notna(city) else (None, None)))
# data['e.x'], data['e.y'] = zip(*data[['e.lon', 'e.lat']].apply(
#     lambda row: lonlat_to_mercator(row['e.lon'], row['e.lat']) if pd.notna(row['e.lon']) and pd.notna(row['e.lat']) else (None, None), axis=1))
data['e.x'], data['e.y'] = zip(*data[['e.longitude', 'e.latitude']].apply(
    lambda row: lonlat_to_mercator(row['e.longitude'], row['e.latitude']), axis=1))

data['e.id'] = 'exhibition_' + data['e.id'].astype(str)
edges = data[['a.id', 'e.id', 'e.startdate']].dropna()

artist_nodes = data[['a.id', 'a.firstname', 'a.lastname', 'a.birthplace', 'a.x', 'a.y']].drop_duplicates()
artist_nodes['artist_label'] = artist_nodes.apply(lambda arow: f"{arow['a.id']}: {arow['a.firstname']} {arow['a.lastname']}", axis=1)
artist_nodes_list = artist_nodes['a.id'].tolist()
artist_labels = artist_nodes['artist_label'].tolist()

exhibition_nodes = data[['e.id', 'e.title', 'e.venue', 'e.x', 'e.y', 'e.country', 'e.city']].drop_duplicates()
exhibition_nodes['exhibition_label'] = exhibition_nodes.apply(lambda erow: f"{erow['e.id']}: {erow['e.title']}", axis=1)
exhibition_nodes_list = exhibition_nodes['e.id'].tolist()
exhibition_labels = exhibition_nodes['exhibition_label'].tolist()

G = nx.Graph()
for artist_id, label in zip(artist_nodes_list, artist_labels):
    G.add_node(artist_id, type='artist', label=label)
for exhibition_id, label in zip(exhibition_nodes_list, exhibition_labels):
    G.add_node(exhibition_id, type='exhibition', label=label)
    
for _, erow in edges.iterrows():
    if erow['a.id'] in artist_nodes_list and erow['e.id'] in exhibition_nodes_list:
        G.add_edge(erow['a.id'], erow['e.id'], year=int(erow['e.startdate']))
    
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
plot = figure(
    title="Artist-Exhibition Dynamic Network", 
    x_axis_type="mercator", 
    y_axis_type="mercator",
    width=1400, 
    height=600, 
    tools="pan,wheel_zoom,reset,save", 
    tooltips="@label"
    )
plot.add_tile(tile_provider)

network_graph = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

node_source = network_graph.node_renderer.data_source
node_colors = []
artist_color_map = {}

artist_palette = [mcolors.to_hex(color) for color in cm.get_cmap('tab20').colors if mcolors.to_hex(color) != '#F4C2C2']

for i, artist in enumerate(artist_nodes_list):
    artist_color_map[artist] = artist_palette[i % len(artist_palette)]

for node_type, node_id in zip(node_source.data['type'], node_source.data['index']):
    if node_type == 'artist':
        node_colors.append(artist_color_map.get(node_id, 'gray'))
    elif node_type == 'exhibition':
        node_colors.append('#F4C2C2') 
        
node_source.data['colors'] = node_colors

graph_labels = []
for node_type, node_id in zip(node_source.data['type'], node_source.data['index']):
    if node_type == 'artist':
        label = G.nodes[node_id]['label']
        graph_labels.append(label)
    else:
        graph_labels.append(str(node_id))

node_source.data['label'] = graph_labels

node_positions_x = []
node_positions_y = []

for node_type, node_id in zip(node_source.data['type'], node_source.data['index']):
    if node_type == 'artist':
        artist_node = artist_nodes[artist_nodes['a.id'] == node_id]
        if not artist_node.empty:
            node_positions_x.append(artist_node.iloc[0]['a.x'])
            node_positions_y.append(artist_node.iloc[0]['a.y'])
        else:
            node_positions_x.append(None)
            node_positions_y.append(None)
    elif node_type == 'exhibition':
        exhibition_node = exhibition_nodes[exhibition_nodes['e.id'] == node_id]
        if not exhibition_node.empty:
            node_positions_x.append(exhibition_node.iloc[0]['e.x'])
            node_positions_y.append(exhibition_node.iloc[0]['e.y'])
        else:
            node_positions_x.append(None)
            node_positions_y.append(None)

node_source.data['x'] = node_positions_x
node_source.data['y'] = node_positions_y

layout_dict = {}
for node_type, node_id, x, y in zip(node_source.data['type'], node_source.data['index'], node_positions_x, node_positions_y):
    if x is not None and y is not None:
        layout_dict[node_id] = (x, y)

static_layout = StaticLayoutProvider(graph_layout=layout_dict)

network_graph.layout_provider = static_layout
network_graph.node_renderer.data_source.data['x'] = node_positions_x
network_graph.node_renderer.data_source.data['y'] = node_positions_y


network_graph.node_renderer.glyph = Circle(radius=30000, fill_color='colors', line_color='black', line_width=0)
network_graph.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.6, line_width=1)

edge_source = network_graph.edge_renderer.data_source
edge_source.data['year'] = [G.edges[edge]['year'] for edge in G.edges]

initial_edge_filter = [year <= 1905 for year in edge_source.data['year']]
edge_filter = BooleanFilter(initial_edge_filter)
edge_view = CDSView(source=edge_source, filters=[edge_filter])

network_graph.edge_renderer.view = edge_view

node_types = node_source.data['type'] 
initial_node_filter = [True if node_types[i] == 'artist' else False for i in range(len(node_types))]
node_filter = BooleanFilter(initial_node_filter)
node_view = CDSView(source=node_source, filters=[node_filter])

network_graph.node_renderer.view = node_view
plot.renderers.append(network_graph)

legend_items = []
for artist, color in artist_color_map.items():
    if artist in node_source.data['index']:
        label = G.nodes[artist]['label']
        rect = Rect(x=0, y=0, width=10, height=10, fill_color=color, line_color=color)
        legend_source = ColumnDataSource(data=dict(x=[0], y=[0]))
        legend_renderer = plot.circle(x='x', y='y', source=legend_source, 
                                      color=color, size=0)
        
        legend_item = LegendItem(label=label, renderers=[legend_renderer])
        legend_items.append(legend_item)
        
legend = Legend(items=legend_items, location="center")
plot.add_layout(legend, 'right')

plot.axis.visible = False
plot.grid.visible = False

slider = Slider(start=1905, end=1915, value=1905, step=1, title="Year")

slider_callback = CustomJS(args=dict(edge_filter=edge_filter, edge_source=edge_source, node_filter=node_filter, node_source=node_source, slider=slider), code="""
    const year = slider.value;
    const years = edge_source.data['year'];
    const edges_start = edge_source.data['start'];
    const edges_end = edge_source.data['end'];
    const nodes_index = node_source.data['index'];
    const nodes_type = node_source.data['type'];
    
    const edge_visibility = [];
    for (let i = 0; i < years.length; i++) {
        if (years[i] <= year) {
            edge_visibility.push(true);
        } else {
            edge_visibility.push(false);
        }
    }
    edge_filter.booleans = edge_visibility;

    const connected_exhibitions = new Set();
    for (let i = 0; i < edge_visibility.length; i++) {
        if (edge_visibility[i]) {
            connected_exhibitions.add(edges_start[i]);
            connected_exhibitions.add(edges_end[i]);
        }
    }

    const node_visibility = [];
    for (let i = 0; i < nodes_index.length; i++) {
        if (nodes_type[i] === 'artist') {
            node_visibility.push(true);
        } else if (nodes_type[i] === 'exhibition') {
            if (connected_exhibitions.has(nodes_index[i])) {
                node_visibility.push(true);
            } else {
                node_visibility.push(false);
            }
        }
    }
    node_filter.booleans = node_visibility;

    edge_source.change.emit();
    node_source.change.emit();
""")
slider.js_on_change('value', slider_callback)

image_path = "./wordcloud.png"
image_div = Div(text=f"<img src='{image_path}' style='width:80%; height:auto;'>")

layout = column(plot, slider, image_div)

output_file(output_path)
show(layout)


C:\Users\Tingyu\AppData\Local\Temp\ipykernel_30300\2335018940.py:96: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  artist_palette = [mcolors.to_hex(color) for color in cm.get_cmap('tab20').colors if mcolors.to_hex(color) != '#B2AC88']
